In [10]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [11]:
driver = webdriver.Chrome()

driver.get("https://baseballsavant.mlb.com/")


In [12]:
PLAYER_SEARCH_INPUT = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "player-auto-complete")))

PLAYER_SEARCH_INPUT.clear()

PLAYER_SEARCH_INPUT.send_keys("Michael Lorenzen")

PLAYER_SEARCH_INPUT.send_keys(Keys.RETURN)


In [13]:
GAME_LOGS = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "tab_gamelogs")))

GAME_LOGS.click()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7D2087072+63090]
	(No symbol) [0x00007FF7D1FF2CC2]
	(No symbol) [0x00007FF7D1E8EC65]
	(No symbol) [0x00007FF7D1ED499D]
	(No symbol) [0x00007FF7D1ED4ADC]
	(No symbol) [0x00007FF7D1F15B37]
	(No symbol) [0x00007FF7D1EF701F]
	(No symbol) [0x00007FF7D1F13412]
	(No symbol) [0x00007FF7D1EF6D83]
	(No symbol) [0x00007FF7D1EC83A8]
	(No symbol) [0x00007FF7D1EC9441]
	GetHandleVerifier [0x00007FF7D24825CD+4238285]
	GetHandleVerifier [0x00007FF7D24BF72D+4488493]
	GetHandleVerifier [0x00007FF7D24B7A0F+4456463]
	GetHandleVerifier [0x00007FF7D21605B6+953270]
	(No symbol) [0x00007FF7D1FFE58F]
	(No symbol) [0x00007FF7D1FF9264]
	(No symbol) [0x00007FF7D1FF939B]
	(No symbol) [0x00007FF7D1FE9BD4]
	BaseThreadInitThunk [0x00007FFE2F357344+20]
	RtlUserThreadStart [0x00007FFE2F8A26B1+33]


In [ ]:
GAME_DATE = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, "2023-04-15")))

original_window = driver.current_window_handle

GAME_DATE.click()

WebDriverWait.until(lambda driver: len(driver.window_handles) > 1)

new_window = [window for window in driver.window_handles if window != original_window][0]
driver.switch_to.window(new_window)

In [ ]:
HIGHLIGHT_GAME = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "flex.highlight-game")))

HIGHLIGHT_GAME.click()

In [ ]:
MATCHUPS = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "button_liveAB_718571")))

MATCHUPS.click()

In [ ]:
DROPDOWN = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "[id*='liveAB-matchups_']")))

select = Select(DROPDOWN)

TEMP_DATA = []

num_options = len(select.options)

for index in range(num_options):
    DROPDOWN = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "[id*='liveAB-matchups_']")))
    select = Select(DROPDOWN)
    
    select.select_by_index(index)
    option_text = select.options[index].text
    
    header = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[starts-with(@id, 'liveABTable_') and contains(@id, '_th-')]")))

    table = header.find_element(By.XPATH, './ancestor::table')

    headers = [th.text for th in table.find_element(By.TAG_NAME, 'thead').find_elements(By.TAG_NAME, 'th')]

    PITCH_DATA = []

    for row in table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr'):
        PITCHES = {headers[index]: cell.text for index, cell in enumerate(row.find_elements(By.TAG_NAME, 'td'))}
        PITCH_DATA.append(PITCHES)

    df_temp = pd.DataFrame(PITCH_DATA, columns=headers)
    df_temp['AT_BAT'] = option_text
    TEMP_DATA.append(df_temp)

SEQUENCE = pd.concat(TEMP_DATA, ignore_index=True)
print(SEQUENCE)

In [ ]:
driver.quit()